# **Tutorial for DICStateField model component**

DICStateField takes the displacement history as an input to the calculation of the 
strain, damage and state fields on an interpolated grid in the **load-space-time** databasis.

In [16]:
%matplotlib widget
import matplotlib.pylab as plt
import numpy as np
from bmcs_shear.api import DICGrid, DICStateFields

The input to the simulation is the `DICGrid` instance providing the history of the displacement field for a grid of markers. 

In [17]:
dic_grid = DICGrid(U_factor=100, dir_name='B7_TV1', t=1)
dic_grid.read_beam_design()

An instance of the state field model component is constructed as

In [18]:
dsf = DICStateFields(dic_grid=dic_grid)

The displacement-driven simulation of the the damage evolution using an anisotropic damage model is started using the `eval` method

In [19]:
dsf.eval()

The calculated fields can then be rendered using the `interact` methods

In [20]:
dsf.interact()

## Grid of finite element quadrature points

The calculation of the strain, stress and damage fields using the finite elements returns the values at the quadrature points of the rectangles. The global positions of these points are available via the property attribute `X_fe_MNa`. As an example, the corner point coordinates can be accessed as

In [21]:
dsf.X_fe_MNa[(0, 0, -1, -1),(0, -1, 0, -1), :]

array([[  16.64914704,   25.64914704],
       [  16.64914704,  302.35085296],
       [1261.35085296,   25.64914704],
       [1261.35085296,  302.35085296]])

Note that the grid `X_fe_MNa` has irregular spacing given by the optimal positioning of the quadrature points within the bilinear finite elements. This can be seen by slicing the first four markers along the first horizontal row $J=0$ and printing the $x$ coordinate $a=0$

In [22]:
dsf.X_fe_MNa[:4, 0, 0]

array([16.64914704, 29.35085296, 38.64914704, 51.35085296])

A regular interpolation grid is provided within the frame covered by $X_{MNa}$ as $X^\mathrm{ipl}_{MNa}$. The same slice along the four markers of the bottom row renders positions with constant spacing as 

In [23]:
dsf.X_ipl_MNa[:4, 0, 0]

array([16.64914704, 27.47264013, 38.29613323, 49.11962632])

## Strain field

The strain field at ultimate load arranged in correspondence to the $M, N$ grid is obtained using the `eps_fields` property, which delivers a tuple with the fields
 - `eps_Emab` - strain tensor $\varepsilon_{ab}$ with with $a,b \in (0,1,2)$ iin each quadrature point $m \in (0, 1, 2, 3)$ of an element $E \in (0, n_E)$ 
 - `eps_MNab` - strain tensor $\varepsilon_{ab}$ with with $a,b \in (0,1,2)$ iin each quadrature grid point with global horizontal index $M \in n_M$ and vertical index $n_N$
 - `eps_MNa` - principal strain $\varepsilon_a$ with $a \in (0,1,2)$ in each quadrature grid point with global horizontal index $M \in n_M$ and vertical index $n_N$
 - `max_eps_MN` - maximal strain in each quadrature grid point with global horizontal index $M \in n_M$ and vertical index $n_N$

In [24]:
eps_Emab, eps_MNab, eps_MNa, max_eps_MN = dsf.eps_fe_fields

The principal stresses in the corner points are obtained as

In [25]:
eps_MNa[(0, 0, -1, -1),(0, -1, 0, -1), :]

array([[-3.43199881e-04, -2.50283043e-04],
       [-5.90662766e-04, -8.85815831e-05],
       [ 3.45449734e-04,  6.62150083e-05],
       [-2.20270959e-04,  3.00748522e-08]])

The maximum and minimum principal strain values are obtained using numpy.max method 

In [26]:
np.max(eps_MNa), np.min(eps_MNa)

(0.0349158315527906, -0.006383017336456406)

The history of strain tensor is accessible via the four dimensional array

In [27]:
dsf.eps_fe_TMNab[-1,0,0,:]

array([[-3.11149976e-04,  4.41676289e-05],
       [ 4.41676289e-05, -2.82332948e-04]])

### Strain interpolator within the load-space domain

To obtain the strain tensor at an arbitrary point of the load-space domain an interpolater over the data points provided for the indexes $T, M, N$ in load, horizontal and vertical dimensions named `f_eps_ipl_txy([t, x, y])` with $t \in (0,1)$.

In [28]:
dsf.f_eps_fe_txy(np.array([0.3, 100, 30]))

array([[[-4.47210744e-04, -7.71511562e-05],
        [-7.71511562e-05, -3.45282131e-04]]])

# Interpolators

Interpolators are provided in form of properties
- The interpolation of the displacements is done on the `DIC` grid with indexes denoted $I,J$.  
- Interpolation of the strain, stress and damage fields is done on the `FE` quadrature grid $M, N$.
- Interpolaters on a finer, interpolated regular grid `IPL` with indexes $M, N$.